# Docker
* https://www.docker.com/
* [Reference documentation](https://docs.docker.com/reference/)

books:
* (Docker A) 浙江大学SEL实验室. **Docker容器与容器云**. 2016-10, 2. edition. 人民邮电出版社.


# Concepts

CLI
- docker
- compose
- dockerd

API
- Docker Engine API
- Docker Hub API
- Extensions API
- Registry API

File formats
- Dockerfile
- Compose file

build checks

> BuildKit has built-in support for analyzing your build configuration based on a set of pre-defined rules for enforcing Dockerfile and building best practices. Adhering to these rules helps avoid errors and ensures good readability of your Dockerfile.
>
> Build checks are supported in:
> 
> - Buildx version 0.15.0 and later
> - docker/build-push-action version 6.6.0 and later
> - docker/bake-action version 5.6.0 and later

# Action

* [15 Common Docker Mistakes (And How to Avoid Them)](https://blog.stackademic.com/15-common-docker-mistakes-and-how-to-avoid-them-525b803d00f9): [cn](https://mp.weixin.qq.com/s/UVTW1RCvGHyxZySxrCL0CQ)

错误 #1：构建庞大臃肿的镜像
- 使用`*-alpine`镜像, 使用`builder`

错误 #2：以 Root 身份运行所有内容
```dockerfile
# Create a non-root user
RUN addgroup -g 1001 -S appgroup && \
    adduser -S appuser -u 1001 -G appgroup

# Set ownership
COPY --chown=appuser:appgroup . /app

# Switch to non-root user
USER appuser
```

错误 #3：硬编码配置值
- 使用`ENV`, `ARG`

错误 #4：忽略层缓存
- 把经常变化的指令放在最后

错误 #5：不使用 .dockerignore

错误 #6：单点故障的健康检查
```shell
HEALTHCHECK --interval=30s --timeout=3s --start-period=60s --retries=3 \
  CMD curl -f http://localhost:3000/health || exit 1
```

错误 #7：混合构建时和运行时的依赖项
- 使用多阶段构建: `COPY --from=builder ...`

错误 #8：糟糕的密钥管理
- 使用环境变量文件

错误 #9：忽略日志管理
- `json-file` driver: `docker run --log-driver=json-file --log-opt max-size=10m --log-opt max-file=3 myapp`
- 使用`fluentd` driver

错误 #10：没有针对 Kubernetes 进行优化
- 优雅处理关闭信号
- Kubernets Deployment: `resources`, `livenessProbe`, `readinessProbe`, `lifecycle.preStop`

错误 #11：版本标签问题
- 不要使用`latest`标签

错误 #12：网络安全漏洞
- 使用`networks`, 不使用默认Docker网络

错误 #13：跳过容器扫描
- `docker scan`, `docker build --scan ...`
- 其他工具: 弱点/漏洞扫描trivy, 应用依赖snyk, 静态分析clairctl 

错误 #14：低效的多平台构建
- 区分构建平台和目标平台

错误 #15：资源限制不足
- `docker run -m 512m --cpus="0.5" myapp`
- docker compose: `deploy.resources.limits/reservations`
- 查看使用情况: `docker stats`, 使用`google/cadvisor`


## Version

```shell
$ docker version
Client:
 Cloud integration: v1.0.35-desktop+001
 Version:           24.0.5
 API version:       1.43
 Go version:        go1.20.6
 Git commit:        ced0996
 Built:             Fri Jul 21 20:36:24 2023
 OS/Arch:           windows/amd64
 Context:           default

Server: Docker Desktop 4.22.1 (118664)
 Engine:
  Version:          24.0.5
  API version:      1.43 (minimum version 1.12)
  Go version:       go1.20.6
  Git commit:       a61e2b4
  Built:            Fri Jul 21 20:35:45 2023
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.6.21
  GitCommit:        3dce8eb055cbb6872793272b4f20ed16117344f8
 runc:
  Version:          1.1.7
  GitCommit:        v1.1.7-0-g860f061
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0
```

```shell
$ docker buildx version
github.com/docker/buildx v0.11.2-desktop.1 986ab6afe790e25f022969a18bc0111cff170bc2
```

# Docker网络

> (Docker A) P.110

驱动:
* bridge - 默认
  * 将Docker容器连接到Docker网桥上, 与外界通信使用NAT.
* host
  * 不创建独立的network namespace, Docker容器与宿主机共用一个network namespace.
* overlay
  * 采用VXLAN方式: 隧道技术, 将一种协议包装在另一个写一种传输, 用于连接使用不同网络而被隔离的主机.
* remote
  * 调用用户的网络驱动插件.
* null
  * Docker容器有自己的network namespace, 未进行网络配置, 需要用户添加网卡,配置IP等.
